In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from pprint import pprint
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats

In [ ]:
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions

FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions


PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions

In [ ]:
hand_model_path = './models/hand_landmarkr_full.task'
face_model_path = './models/face_landmarker.task'
pose_model_path = './models/pose_landmarker_heavy.task'

In [ ]:
handOptions = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=hand_model_path),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=2)

faceOptions = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=face_model_path),
    running_mode=VisionRunningMode.VIDEO)

poseOptions = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=pose_model_path),
    running_mode=VisionRunningMode.VIDEO)

In [ ]:
def initModels():
    handLandmarker=HandLandmarker.create_from_options(handOptions)
    faceLandmarker=FaceLandmarker.create_from_options(faceOptions)
    poseLandmarker=PoseLandmarker.create_from_options(poseOptions)
    return handLandmarker,faceLandmarker,poseLandmarker
handLandmarker=HandLandmarker.create_from_options(handOptions)
faceLandmarker=FaceLandmarker.create_from_options(faceOptions)
poseLandmarker=PoseLandmarker.create_from_options(poseOptions)
# initModels()

In [ ]:
def draw_landmarks_on_image(rgb_image, pose_detection_result,hand_detection_result):
  pose_landmarks_list = pose_detection_result.pose_landmarks
  
  hand_landmarks_list = hand_detection_result.hand_landmarks
  handedness_list = hand_detection_result.handedness
  
  annotated_image = np.copy(rgb_image)

  MARGIN = 10  # pixels
  FONT_SIZE = 1
  FONT_THICKNESS = 1
  HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
        annotated_image,
        pose_landmarks_proto,
        solutions.pose.POSE_CONNECTIONS,
        solutions.drawing_styles.get_default_pose_landmarks_style())
    
  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
        annotated_image,
        hand_landmarks_proto,
        solutions.hands.HAND_CONNECTIONS,
        solutions.drawing_styles.get_default_hand_landmarks_style(),
        solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)
  return annotated_image

In [ ]:
def save_data(pose_landmarket_result, hand_landmarker_result, name=None,npReturn=False):
    
    if len(pose_landmarket_result.pose_landmarks) > 0:
        finalPoseData: np.ndarray = np.array([[i.x, i.y, i.z]
                                              for i in pose_landmarket_result.pose_landmarks[0]]).flatten()
    else:
        finalPoseData = np.zeros((99,))
    finalLeftHandData = np.zeros((63,))
    finalRightHandData = np.zeros((63,))
    for idx in range(len(hand_landmarker_result.hand_landmarks)):
        
        if hand_landmarker_result.handedness[idx][0].category_name == "Left":
            finalLeftHandData=np.array([[i.x, i.y, i.z] for i in hand_landmarker_result.hand_landmarks[[idx][0]]]).flatten()
        else:
            finalRightHandData = np.array([[i.x, i.y, i.z] for i in hand_landmarker_result.hand_landmarks[[idx][0]]]).flatten()

    if npReturn:
        return np.concatenate([ finalPoseData, finalLeftHandData, finalRightHandData])
    np.save(name, np.concatenate([ finalPoseData, finalLeftHandData, finalRightHandData]))

In [ ]:
cap = cv2.VideoCapture(0)
prev_frame_time = 0
new_frame_time = 0
startTime=time.time()
handLandmarker, faceLandmarker, poseLandmarker = initModels()
while cap.isOpened():
    new_frame_time = time.time()

    ret, frame = cap.read()

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    hand_landmarker_result = handLandmarker.detect_for_video(
        mp_image,int((time.time()-startTime)*1000))
    pose_landmarket_result = poseLandmarker.detect_for_video(
        mp_image,int((time.time()-startTime)*1000))

    image = cv2.cvtColor(draw_landmarks_on_image(
        mp_image.numpy_view(), pose_landmarket_result,hand_landmarker_result), cv2.COLOR_RGB2BGR)

    # pprint(hand_landmarker_result.handedness)
    save_data(pose_landmarket_result, hand_landmarker_result, "testSave.npy")

    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    cv2.putText(image, f"{fps:1f}", (15, 50),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
    cv2.imshow('OpenCV Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
sign=['control','yes','no','thankYou','hello','iLoveYou','peace','please',]

In [ ]:
DATA_PATH = os.path.join('data')
actions = np.array(sign)
no_sequences = 100
# sequence_length = 15

In [ ]:
for action in actions:
    try:
        os.mkdir(DATA_PATH+f'/{action}')
    except:
        ...

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
def collectData(action, length=no_sequences,start=0):
    handLandmarker, faceLandmarker, poseLandmarker = initModels()
    startTime = time.time()
    for frame_num in range(length):
        ret, frame = cap.read()

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                            data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        hand_landmarker_result = handLandmarker.detect_for_video(
            mp_image, int((time.time()-startTime)*1000))
        pose_landmarket_result = poseLandmarker.detect_for_video(
            mp_image, int((time.time()-startTime)*1000))

        image = cv2.cvtColor(draw_landmarks_on_image(
            mp_image.numpy_view(), pose_landmarket_result, hand_landmarker_result), cv2.COLOR_RGB2BGR)

        if frame_num == 0:
            cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4, cv2.LINE_AA)
            cv2.putText(image, 'Collecting frames for {} Frame No {}'.format(action, frame_num), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)
            cv2.waitKey(500)
        else:
            cv2.putText(image, 'Collecting frames for {} Frame No {}'.format(action, frame_num), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

        npy_path = os.path.join(
            DATA_PATH,action, str(start+frame_num))
        save_data(pose_landmarket_result,
                    hand_landmarker_result, npy_path, )
        # Break gracefully
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

In [ ]:
cap = cv2.VideoCapture(0)
collectData('peace',500,0)
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(sign)
cap = cv2.VideoCapture(0)
for action in actions:
    input()
    collectData(action,500,0)
cap.release()
cv2.destroyAllWindows()

In [ ]:
label_map = {label: num for num, label in enumerate(actions)}.copy()
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    
    for frame_num in range(500):
        res = np.load(os.path.join(DATA_PATH, action, str(frame_num)+'.npy',))
        
        sequences.append(res)
        labels.append(label_map[action])
np.array(sequences).shape, np.array(labels).shape

In [ ]:
X = np.array(sequences)
Y = to_categorical(labels).astype(int)
X.shape,Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [ ]:
X_train.shape[1:],actions.shape[0]

In [ ]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
# model.add(LSTM(64, return_sequences=True,
#           activation='relu', input_shape=X_train.shape[1:]))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(256, activation='relu' , input_shape=X_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
res = model.predict(X_test)
actions[np.argmax(res[4])], actions[np.argmax(Y_test[4])]

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(Y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
# np.expand_dims(X_test[0], axis=0)

In [ ]:
model.predict(X_test)

In [ ]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]


def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40),
                      (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return output_frame

In [ ]:
try:
    cap = cv2.VideoCapture(0)
    prev_frame_time = 0
    new_frame_time = 0
    startTime = time.time()
    handLandmarker, faceLandmarker, poseLandmarker = initModels()
    while cap.isOpened():
        new_frame_time = time.time()

        ret, frame = cap.read()

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                            data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        hand_landmarker_result = handLandmarker.detect_for_video(
            mp_image, int((time.time()-startTime)*1000))
        pose_landmarket_result = poseLandmarker.detect_for_video(
            mp_image, int((time.time()-startTime)*1000))

        image = cv2.cvtColor(draw_landmarks_on_image(
            mp_image.numpy_view(), pose_landmarket_result, hand_landmarker_result), cv2.COLOR_RGB2BGR)

        # pprint(hand_landmarker_result.handedness)
        concatArray=save_data( pose_landmarket_result,
                hand_landmarker_result, npReturn=True)
        
        
        
        predict = model.predict(np.expand_dims(concatArray, axis=0))
        predictF=actions[np.argmax(predict)]
        cv2.putText(image, f"{predictF}", (15, 50),
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        cv2.putText(image, f"{(np.max(predict))}", (15, 410),
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        # cv2.putText(image, f"{predict[0][1]}", (15, 440),
        #             cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        # cv2.putText(image, f"{predict[0][2]}", (15, 470),
        #             cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)

        

        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        # cv2.putText(image, f"{fps:1f}", (15, 50),
        #             cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
except Exception as e:
    cap.release()
    cv2.destroyAllWindows()
    raise e

In [29]:
model.save("modelv2")

INFO:tensorflow:Assets written to: modelv2/assets


INFO:tensorflow:Assets written to: modelv2/assets


In [28]:
actions

array(['control', 'yes', 'no', 'thankYou', 'hello', 'iLoveYou', 'peace',
       'please'], dtype='<U8')